# Assigment 2: PyTorch Model Zoo

The [PyTorch "Model Zoo"](https://pytorch.org/vision/stable/models.html) provides a large number of pre-trained CNN models and vision [data sets](https://pytorch.org/vision/stable/datasets.html)...

In [1]:
#imports
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
#transform input data (image) to tensor
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#set batch size
batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Assignment 1:
Load a "*ResNet18*" from the torchvision model zoo and train it for 10 epochs

In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torch.optim.lr_scheduler import MultiStepLR

resnet18 = models.resnet18().cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=0.8)

In [9]:
running_loss = 0.0
for epoch in range(10):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].cuda(),data[1].cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:    # every 1000 mini-batches...
            print(running_loss / 1000)
            running_loss = 0.0
print('Finished Training')

2.399986220359802
2.2446963996887206
2.1552910045385363
2.065370506644249
2.0277747955322267
1.978003435432911
1.9158456836938857
1.878657375395298
1.872189746916294
1.8434651392698287
1.774636933028698
1.729168035030365
2.5146504380702974
1.7044165627956391
1.6109072524309158
1.5584261916577815
1.5322452142834664
1.5346636624336243
1.483274699240923
1.4459562092125415
1.419438724130392
1.4593617942333221
1.4860361000895501
1.4004083817899227
2.07477248480916
1.3239265755861997
1.364003390058875
1.3332247073054313
1.2863648433834314
1.2725022315084935
1.3216034683287143
1.2431508453935385
1.2925070890933275
1.281741875141859
1.2509081365019084
1.2464598045796156
1.8694212624281645
1.2024922107458114
1.1801948217153548
1.179184256747365
1.1635382577106357
1.256278126820922
1.1977040833160282
1.2068189116865395
1.1435547482296824
1.1487651003822685
1.1498026980161666
1.1424148351326584
1.7140819362103938
1.0973260325789451
1.1011446002461016
1.144449584722519
1.0573275154978037
1.0941442

In [ ]:
correct = 0
total = 0

# since we're not training, we don't need to calculate the gradients for our outputs
resnet18.eval()
    
for data in testloader:
        images, labels = data[0].cuda(),data[1].cuda()
        # calculate outputs by running images through the network
        outputs = resnet18(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

## Assigment 2:
Load a **pre-trained** (on ImageNet) "*ResNet18*" from the torchvision model zoo and *fine-tune* it for ten epochs

In [11]:

resnet18 = models.resnet18(pretrained=True).cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=0.0001)
scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=0.8)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [12]:
running_loss = 0.0
for epoch in range(10):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].cuda(),data[1].cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:    # every 1000 mini-batches...
            print(running_loss / 1000)
            running_loss = 0.0
print('Finished Training')

3.670693365097046
2.0772690289318563
1.9270752306580543
1.824203920930624
1.779035026460886
1.7579060722291469
1.6931463679671288
1.729331436932087
1.6469624396264553
1.6083205479681493
1.5654194400906563
1.555639343649149
2.3097414421439173
1.4931820391118527
1.4615098820626735
1.4309579722881318
1.4300172598063945
1.4303601144254208
1.4216811482310294
1.4200156029611826
1.480567331403494
1.4010302763283253
1.3905338222682475
1.3551061437278986
1.9765083717405796
1.3291582031846045
1.2925981197357177
1.2622150317132472
1.3014054619818927
1.2520530562326313
1.2464557464420796
1.2761108380258084
1.194513883203268
1.2176472670733929
1.2030734455287457
1.1886240476667882
1.8457340027689935
1.13375129686296
1.1290090080499648
1.1557886466160416
1.1540998756736518
1.1979733504951
1.1349831653088331
1.117477264985442
1.0903621063753963
1.0814543593525887
1.1135625617951155
1.087392873033881
1.5470692756772042
1.0734549308866261
1.0357467694953084
1.0561569638326764
1.0334854445978998
1.02738

In [15]:
correct = 0
total = 0

resnet18.cuda()

# since we're not training, we don't need to calculate the gradients for our outputs
resnet18.eval()
    
for data in testloader:
        images, labels = data[0].cuda(),data[1].cuda()
        # calculate outputs by running images through the network
        outputs = resnet18(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 72 %
